## 10.1 Vector : 사용자 정의 시퀀스형

- 요소들을 실수형 배열에 저장하고, 벡터가 균일 시퀀스처럼 작동하게 만들기 위해 필요한 메서드들 구현

## 10.2 Vector 버전 #1 : Vector2d호환

#### 예제 10-2 vector_v1.py : vector2d_v1.py 에서 유도

In [10]:
from array import array
import reprlib
import math

class Vector:
    typecode = "d"
    
    def __init__(self, components):
        self._components = array(self.typecode, components) # 보호된 객체 속성인 self._components는 벡터 요소를 배열로 저장
        
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 self._components에 대한 반복자를 반환
    
    def __repr__(self):
        components = reprlib.repr(self._components) # self._components를 제한된 길이로 표현하기 위해
        # [0.0, 1.0, 2.0, 3.0, 4.0, ...]  형태로 출력
        components = components[components.find('['):-1]
        # array('d', 와 마지막에 나오는 괄호 제거
        # Vector 내부에 배열을 사용한다는 구현 내용을외부에 노출시키고 싶지 않았기 때문에
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return(bytes([ord(self.typecode)]) + bytes(self._components)) # self._components에서 바로 bytes 객체를 생성
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [6]:
reprlib.repr('supercalifragilisticexpialidocious')

"'supercalifra...xpialidocious'"

In [7]:
reprlib.repr(set('supercalifragilisticexpialidocious'))

"{'a', 'c', 'd', 'e', 'f', 'g', ...}"

#### 예제 10-1 Vector.\__init\__()과 Vector.\__repr\__() 테스트

In [11]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [12]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [14]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

<hr>

## 10.3 프로토콜과 덕 타이핑
- 객체지향 프로그래밍에서 프로토콜은 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스
    - 파이썬 시퀀스 프로토콜은 \__len\__()과 \__getitem\__() 메서드를 동반
    - 표준 시그너처와 의미에 따라 이 메서드들을 구현하는 어떠한 클래스도 시퀀스가 필요한 곳에 사용 될 수 있음

#### 예제 10-3 [예제 1-1]의 코드

In [9]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

- 시퀀스 프로토콜을 구현하므로 파이썬에서 제공하는 여러 기능 활용 가능 
- 코드 어디에도 시퀀스 프로토콜을 따른다고 정의한 곳은 없지만 이 클래스가 시퀀스처럼 **동작** 하기 때문에 시퀀스
- 프로토콜이 비공식적이면 강제로 적용되는 사항이 아니므로클래스가 사용되는 특정 환경에 따라일부만 구현 가능

## 10.4 Vector 버전 #2 : 슬라이스 가능한 시퀀스

In [32]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]

In [33]:
v1 = Vector([3, 4, 5])
print(len(v1))
print(v1[0], v1[-1])

3
3.0 5.0


In [36]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

- Vector를 슬라이싱해서 생성된 평범한 배열을 Vector의 기능을 상실
- Vector를 슬라이싱해서 Vector 객체를 생성하려면, 슬라이싱 연산으 ㄹ배열에 위임하면 안되며, \__getitem\__() 메서드가 받은 인수를 분석해서 제대로 처리

### 10.4.1 슬라이싱의 작동 방식

#### 예제 10-4 \__getitem\__()과 slice의 동작 확인

In [37]:
class Myseq:
    def __getitem__(self,index):
        return index

In [42]:
s = Myseq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])# []안에 ,가 들어가면 __getitem__()이 이 튜플을 받음
print(s[1:4:2, 7:9]) # 튜플 안에 여러 슬라이스 객체가 들어 있을 수도 있음

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


#### 예제 10-5 slice 클래스의속성 조사

In [45]:
print(slice) # 내장된자료형
print(dir(slice)) # start, step, stop 속성과 indices 속성 존재

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']


In [47]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



- indieces는 주어진 길이의 시퀀스 경계 아에 들어가도록 조정된 0이나 양수인 start, stop, stirde로 구성된 정규화된 튜플을 생성

In [48]:
slice(None, 10, 2).indices(5) # 'ABCDE'[:10:3]는 'ABCDE'[0:5:2]

(0, 5, 2)

In [50]:
slice(-3, None, None).indices(5) # 'ABCDE'[-3:] 는 'ABCDE'[2:5:1]

(2, 5, 1)

### 10.4.2 슬라이스를 인식하는 \__getitem\__()

#### 예제 10-6 vector_v2.py 

In [51]:
from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    

    
    #########################################################################
    
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)  # 객체의 클래스 (즉, Vecto 를 불러옴)
        if isinstance(index, slice): # index 인수가 slice 이면 
            return cls(self._components[index]) # _components 배열의 슬라이스로부터 Vector 클래스 생성자를 이용해 Vector 객체 생성
        elif isinstance(index, numbers.Integral): # index 인수가 int형 등 정수형이면
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))



- isinstance()를 많이 사용한다는 것은 객체지향 설계까 잘못되었다는 것을 나타냄
- \__getitem\__()에서 슬라이스를 처리하는 경우에는 정당화 될 수 있음

In [52]:
'ABC'[1,2]

<>:1: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
<ipython-input-52-9df85be745b9>:1: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
  'ABC'[1,2]


TypeError: string indices must be integers

#### 예제 10-7

In [55]:
v7 = Vector(range(7))
v7[-1] # 정수형 인덱스는 단 한 요소의 값을 실수형으로 반환

6.0

In [56]:
v7[1:4] # 슬라이스 인덱스는 Vector를 새로 만듦

Vector([1.0, 2.0, 3.0])

In [57]:
v7[-1:] # 길이가 1인 슬라이스도 Vector 객체 생성

Vector([6.0])

In [58]:
v7[1,2] # Vector는 다차원 인덱싱을 지원하지 않으므로 인덱스나 슬라이스로 구성된 튜플은 에러를 발생

TypeError: Vector indieces must be integers

## 10.5 Vector 버전 #3 : 동적 속성 접근
- Vector2d에서 Vector로 진화하면서 v.x, v.y 처럼 벡터 요소를 이름으로 접근하는 능력 상실

In [60]:
v = Vector(range(10))
v.x

AttributeError: 'Vector' object has no attribute 'x'

In [61]:
v.y, v.z, v.t

AttributeError: 'Vector' object has no attribute 'y'

#### 예제 10-8 vector_v3.py

In [62]:
from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    

    
    #########################################################################
    
    def __len__(self):
        return len(self._components)

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))



#### 예제 10-9 부적절한 동작. v.x에 값을 할당하더라도 에러가 발생하지는 않지만 v.x 값이 일관성 없게 된다

In [63]:
v = Vector(range(10))
v.x

0.0

In [66]:
v.y, v.z, v.t

(1.0, 2.0, 3.0)

In [67]:
v = Vector(range(5))

In [68]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [69]:
v.x

0.0

In [70]:
v.x = 10

In [71]:
v.x

10

In [72]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

- \__getattr\__() 매서드가 작동하는 방식 때문에 발생
- 파이썬은 해당 이름의 속성을 찾지 못할 때 최후 수단으로 \__getattr\__() 호출
- 그러나 v.x = 10 문장으로 x 값에 10을 할당할 때, v객체에 x 속성이 추가되므로 , 더이상 v.x 값을 가져오기 위해 \__getattr\__() 호출하지 않음
- 인터프리터는 단지 v.x에 바인딩된 값 10을 반환

#### 예제 10-10 vector_v3.py : vector 클래스에 추가된 \__setattr\__() 메서드

In [73]:
from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    

    
    #########################################################################
    
    def __len__(self):
        return len(self._components)

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 단일 문자 속성명에 대해 특별한 처리를 함
            if name in cls.shortcut_names:  # name이 x, y, z, t 중 하나이면 구체적인 에러메세지
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3> # 그 외 소문자이면
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else: # 그렇지 않으면
                error = ''  # <4>
            if error:  # <5># error 안에 어떤 문자가 들어있으면 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 호출

- super() 함수는 슈퍼클래스의 메서드에 동적으로 접근할 수 있는 방법을 제공
- 파이썬과 같이 다중 상속을 지원하는 동적 언어에서 필수적인 기능

## 10.6 Vector 버전 #4 : 해싱 및 더 빠른 ==
- \__hash\__() 매서드를 구현하면 기존 \__eq\__() 메서드와 함께 Vector 객체를 해시할 수 있음

In [75]:
2*3*4*5

120

In [76]:
import functools
functools.reduce(lambda a, b,: a*b, range(1, 6))

120

#### 예제 10-11 0에서 5까지 정수를 XOR로 누적 계산하는 세가지 방법

In [77]:
# 1
n = 0
for i in range(6):
    n ^= i
n

1

In [78]:
# 2
import functools
functools.reduce(lambda a, b: a^b, range(6))

1

In [79]:
# 3
import operator
functools.reduce(operator.xor, range(6))

1

#### 예제 10-12 vector_v4.py : vector_v3.py에 두개의 임포트 문과 \__hash\__()메서드 추가

In [81]:
from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    


    def __len__(self):
        return len(self._components)

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 단일 문자 속성명에 대해 특별한 처리를 함
            if name in cls.shortcut_names:  # name이 x, y, z, t 중 하나이면 구체적인 에러메세지
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3> # 그 외 소문자이면
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else: # 그렇지 않으면
                error = ''  # <4>
            if error:  # <5># error 안에 어떤 문자가 들어있으면 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 호출

        
        
    #########################################################################
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0) # xor 함수와 hashes를 전달해서 해시값들의 XOR을 구한다, 세번째 인수는 초깃값

- reduce()를 사용할 때는 세 번째 인수를 전달해서 reduce(<함수>, <반복형>, <초기값>) 형태로 호출
- <초기값>은 시퀀스가 비어있을 때 반환되는 값이며, 함수에 대한 항등원을 사용해야 함
- TypeError 방지

In [84]:
from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    


    def __len__(self):
        return len(self._components)

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 단일 문자 속성명에 대해 특별한 처리를 함
            if name in cls.shortcut_names:  # name이 x, y, z, t 중 하나이면 구체적인 에러메세지
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3> # 그 외 소문자이면
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else: # 그렇지 않으면
                error = ''  # <4>
            if error:  # <5># error 안에 어떤 문자가 들어있으면 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 호출

        
        
    #########################################################################
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)

- Python3에서는 map()을 느긋하게 수행하여 필요할 때 결과를 산출하는 제너레이터를 생성

- 단지 튜플형의 \__eq\__() 메서드를 적용하기 위해 피연산자 전체를 복사해서 튜플 2개를 만듦
- 다차원의 배열인 경우에는 효율이 떨이짐

#### 예제 10-13 더 효율적으로 비교하기 위해 for 루프 안에 zip 사용

In [ ]:
from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    


    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))
    
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 단일 문자 속성명에 대해 특별한 처리를 함
            if name in cls.shortcut_names:  # name이 x, y, z, t 중 하나이면 구체적인 에러메세지
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3> # 그 외 소문자이면
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else: # 그렇지 않으면
                error = ''  # <4>
            if error:  # <5># error 안에 어떤 문자가 들어있으면 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 호출

        
        
    #########################################################################
    
    def __eq__(self, other):
        if len(self) != len(other): # 두 객체의 길이가 다르면, 객체가 다름
            return False
        
        for a, b, in zip(self, other): # zip 함수는 반복형 인수의 항목으로 구성된 튜플의 제너레이터를 만듦
            if a != b:
                return False
        return True
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)

- 효율이 높지만 , all() 함수를 사용하면 for 루프와 동일한 계싼을 단 한 줄에 할 수 있음
- 해당 요소간 비교가 모두 True이면 결과도 True
- 비교하는 도중에 다른 요소가 나오면, 즉 비교가 False이면 all()은 바로 False 반환

#### 10-14 zip()과 all() 함수를 이용한 메서드

In [ ]:
from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    


    def __len__(self):
        return len(self._components)

    shortcut_names = 'xyzt'
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))
    
    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 단일 문자 속성명에 대해 특별한 처리를 함
            if name in cls.shortcut_names:  # name이 x, y, z, t 중 하나이면 구체적인 에러메세지
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3> # 그 외 소문자이면
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else: # 그렇지 않으면
                error = ''  # <4>
            if error:  # <5># error 안에 어떤 문자가 들어있으면 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 호출

        
        
    #########################################################################
    
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)

#### 예제 10-15 내장된 zip() 함수의 사용

In [85]:
zip(range(3), 'ABC') # 튜플을 생성하는 제너레이터를 반환

In [86]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [88]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3])) # 반복형 중 어느것이라도 소진되면 경고 메세지 없이 중단

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [89]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1)) # fillvalue를 이용하여 빠진 값을 채워가면서
# 마지막 반복형이 소진될 때 까지 튜플생성 (fillvalue defaul: None)

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

## 10.7 Vector 버전 #5: 포매팅
- 

#### 예제 10-16

In [115]:
from array import array
import reprlib
import math
import functools
import operator
import itertools

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    


    def __len__(self):
        return len(self._components)

    shortcut_names = 'xyzt'

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))
    
    def __getattr__(self, name):
        cls = type(self)  # Vector 클래스 가져오기
        if len(name) == 1:  # name이 한 글자이면 shortcut_names 들 중 하나 일 수 있음
            pos = cls.shortcut_names.find(name)  # 한 글자 name의 위치 찾음
            if 0 <= pos < len(self._components):  # <4>
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  # <5>
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # 단일 문자 속성명에 대해 특별한 처리를 함
            if name in cls.shortcut_names:  # name이 x, y, z, t 중 하나이면 구체적인 에러메세지
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <3> # 그 외 소문자이면
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else: # 그렇지 않으면
                error = ''  # <4>
            if error:  # <5># error 안에 어떤 문자가 들어있으면 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 __setattr__() 호출

        
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)
    
    
    #########################################################################
    
    def angle(self, n):  # 초구면좌표에 대한 공식을 이용해서 특정 좌표에 대한 각 좌표를 계싼
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # 요청에 다라 각좌표를 모두 계산하는 제너레이터 표현식 생성
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        print(self)
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # itertools.chain 함수릴 이용해 크기와 각좌표를 차례대로 반복하는 제너레이터 표현식 생성
            outer_fmt = '<{}>'  # 구면 좌표 출력 설정
        else:
            coords = self
            outer_fmt = '({})'  # 직교좌표 출력 설정
        components = (format(c, fmt_spec) for c in coords)  # 좌표의 각 항목을 요청에 따라 포맷하는 제너레이터 생성
        return outer_fmt.format(', '.join(components))  # <8> 포맷된 요소들을 콤마로 분리해서 꺽쇠 괄호나 괄호 안에 넣음

In [116]:
format(Vector([-1,-1, -1, -1]), 'h') # vector 크기, 각좌표

(-1.0, -1.0, -1.0, -1.0)


'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [118]:
format(Vector([3, 4]))

(3.0, 4.0)


'(3.0, 4.0)'